In [27]:
import numpy as np
import pandas as pd
import re

df = pd.read_pickle('films_1983.pkl')
os = pd.read_pickle('oscar_film_noms.pkl')

In [34]:
oscars = pd.read_csv('oscar_noms_by_year.csv', index_col='name')
baftas = pd.read_csv('bafta_noms_by_year.csv', index_col='name')
globes = pd.read_csv('globe_noms_by_year.csv', index_col='name')

def results(name, awards, year=2020):
    if name in awards.index:
        nom = won = 0
        noms = awards.loc[name].nom[1:-1].split()
        wins = awards.loc[name].won[1:-1].split()
        for y in noms:
            if int(y) < year: nom += 1
        for y in wins:
            if int(y) < year: won += 1
        return nom, won
    else: return 0, 0

def results_person(name, awards, year=2020):
    if name in awards.index:
        nom = won = 0
        nom_entries = awards.filter(like=name, axis=0).nom.values
        noms = ''
        for entry in nom_entries:
            noms = noms + entry[1:-1] + ' '
        noms = noms.split()
        win_entries = awards.filter(like=name, axis=0).won.values
        wins = ''
        for entry in win_entries:
            wins = wins + entry[1:-1] + ' '
        wins = wins.split()
        for y in noms:
            if int(y) < year: nom += 1
        for y in wins:
            if int(y) < year: won += 1
        return nom, won
    else: return 0, 0
    
def all_results(name, year=2020):
    osc = results(name, oscars, year)
    baf = results(name, baftas, year)
    glo = results(name, globes, year)
    return osc[0]+baf[0]+glo[0], osc[1]+baf[1]+glo[1]

def all_results_person(name, year=2020):
    osc = results_person(name, oscars, year)
    baf = results_person(name, baftas, year)
    glo = results_person(name, globes, year)
    return osc[0]+baf[0]+glo[0], osc[1]+baf[1]+glo[1]

def get_rating_person(index, credit, year):
    nom = won = 0
    for name in df.iloc[index][credit]:
        rating = all_results_person(name, year)
        nom += rating[0]
        won += rating[1]
    return nom, won

def get_rating_person_added(index, credit, year):
    nom = won = 0
    for name in df.iloc[index][credit]:
        rating = all_results_person(name, year)
        nom += rating[0]
        won += rating[1]
    return nom + 3*won

def get_rating(index, credit, year):
    nom = won = 0
    for name in df.iloc[index][credit]:
        rating = all_results(name, year)
        nom += rating[0]
        won += rating[1]
    return nom, won

In [35]:
processed = pd.DataFrame(columns=['title', 'director', 'producer', 'cast', 'noms', 'wins'])


In [36]:
for n in df.index:
    d = {}
    d['title'] = df.iloc[n].title
    d['director'] = get_rating_person_added(n, 'director', 1983)
    d['producer'] = get_rating_person_added(n, 'producer', 1983)
    d['cast'] = get_rating_person_added(n, 'cast', 1983)
    
    t = df.iloc[n].title + ' 1983'
    if len(os.query('film == @t').film.values) == 0:
        d['noms'] = 0
        d['wins'] = 0
    else:
        d['noms'] = os[os['film'] == t].noms.values[0]
        d['wins'] = os[os['film'] == t].wins.values[0]
    
    
    processed = processed.append(d, ignore_index=True)
    
    


In [37]:
processed = processed.drop_duplicates()

In [44]:
processed['overall'] = processed.director + processed.producer + processed.cast

In [46]:
processed[processed.overall > 30]

,title,director,producer,cast,noms,wins,overall
3,Local Hero,5,8,23,0,0,36
30,The Thorn Birds,0,0,38,0,0,38
42,The Osterman Weekend,0,0,41,0,0,41
46,The Scarlet and the Black,0,0,52,0,0,52
48,Zelig,36,0,44,2,0,80
64,Yentl,44,44,44,5,1,132
70,To Be or Not to Be,0,10,52,1,0,62
90,Silkwood,22,22,42,5,0,86
96,The King of Comedy,4,0,28,0,0,32
108,Star 80,13,0,20,0,0,33


In [ ]:
#a = ['Columbia', 'Warner Distributors', 'Warner Bros. Pictures (US)']
#True in [x in n for n in a for x in ['Warner', 'Paramount']]

In [ ]:
#int(file.budget[0].replace(',', ''))